In [460]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from time import sleep
import urllib
import os
from bs4 import BeautifulSoup
import requests

In [2]:
options = Options()
# options.add_argument("--disable-blink-features")
# options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

dr = webdriver.Chrome(options=options)

In [11]:
dr.get('https://www.coursera.org/?authMode=login')
dr.find_element_by_xpath('//input[@type = "email"]').send_keys('Bcsf17m521@pucit.edu.pk')
dr.find_element_by_xpath('//input[@type = "password"]').send_keys('Coursera!!4040')
dr.find_element_by_xpath('//input[@type = "password"]').send_keys(Keys.ENTER)

In [471]:
## soup = BeautifulSoup(dr.page_source, "html.parser")
courseTitle = soup.find('h1').text
os.mkdir(courseTitle) 
os.mkdir(f'{courseTitle}/assets')
os.mkdir(f'{courseTitle}/assets/images') 

In [579]:
images_path = f'{courseTitle}/assets/images'.replace('\\', '/')
images_counter = 0

In [487]:
if dr.current_url.find('/welcome') != -1:
    base_url = dr.current_url 
    soup = BeautifulSoup(dr.page_source)
    weeks = [x.text for x in soup.findAll("a", {"data-track-component":"nav_week"})]

In [439]:
for w in weeks:
    dr.get(base_url.replace('welcome', 'week/1'))
    WebDriverWait(dr, 20).until(EC.presence_of_element_located((By.XPATH, '//p[contains(text(), "Discuss the topic here.")]')))
    break

In [440]:
soup = BeautifulSoup(dr.page_source)
temp = soup.find('div', {'class':'horizontal-box named-item-list-title'}).next.next.next
link_to_first_lec_of_week = [x for x in temp][0].contents[0]['href']
dr.get("https://coursera.org{}".format(link_to_first_lec_of_week)) 
WebDriverWait(dr, 20).until(EC.presence_of_element_located((By.XPATH, '//button[contains(text(), "Download")]')))

<selenium.webdriver.remote.webelement.WebElement (session="bc0db738d03041a195b69bfc6bca80e8", element="83cabd32-c0f8-4f6d-9021-05e92b031d11")>

In [302]:
# clicks on navigation contents button if not clicked by default
toggle_nav_but = dr.find_element_by_xpath('//button[@aria-label="Toggle Navigation Panel"]')
if toggle_nav_but.get_attribute('aria-expanded')=='false':
    toggle_nav_but.click()

In [303]:
## expand contents
contents = dr.find_elements_by_xpath('//button[@class="nostyle link-button"]') 
for c in contents: 
    if c.get_attribute('aria-expanded') == 'false':
        c.click() 
sleep(1)

In [124]:
html_form = """<html>
<p>
template
</p>
</html>"""

In [263]:
soup = BeautifulSoup(dr.page_source, "html.parser")
contentNames = [item.text for menus in soup.findAll(class_="rc-LessonItems nostyle")
 for itemList in menus.findAll('li') 
 for item in itemList.findAll('div', {'class':'rc-NavItemName caption-text'})
]

contentLinks = [f"https://coursera.org{item['href']}" for menus in soup.findAll(class_="rc-LessonItems nostyle")
 for itemList in menus.findAll('li') 
 for item in itemList.findAll('a', {'href':True})
]

In [594]:
lecture_elements = dr.find_elements_by_xpath('//div[@class="rc-NavItemIcon"]')

In [595]:
contentTitles = [] 

In [596]:
for el in lecture_elements:
    try:
        el.click()
    except:
        continue
    try:
        WebDriverWait(dr, 4).until(EC.presence_of_element_located((By.XPATH, '//button[@aria-label="Like"]')))
    except:
        sleep(2)
        pass
    sleep(1)
    soup = BeautifulSoup(dr.page_source, "html.parser").find('div', {'class':'item-scroll-container'})
    if soup.find('h3', {'class':'video-name'}) is not None:
        title = f"Video: {soup.find('h3', {'class':'video-name'}).text}"
    elif  soup.find('div', {'class':'reading-title'}) is not None:
        title = f"Reading: {soup.find('div', {'class':'reading-title'}).text}"
    else:
        title = soup.find('h2').text
    
    
    contentTitles.append(title.replace(":","").replace("<","").replace(">",""))
    createLocalHTML(soup, title)

In [597]:
contentTitles

['Video Video Welcome to Class - Dr. Chuck',
 'Reading Reading Welcome to The Class',
 'Video Video Welcome to Python - Guido van Rossum',
 'Reading Help Us Learn More About You!',
 'Reading Welcome to Python 3',
 'Reading Textbook Python for Everybody Exploring Data in Python 3',
 'Reading Submitting Assignments',
 'Video 1.1 - Why Program',
 'Video 1.2 - Hardware Overview',
 'Video 1.3 - Python as a Language',
 'Reading Audio Versions of All Lectures',
 'Video Fun The Textbook Authors Meet @PyCon2015',
 'Video Face to Face Office Hours - Bengaluru, India']

In [615]:
## for "reading" items
soup = BeautifulSoup(dr.page_source, "html.parser").find('div', {'class':'item-scroll-container'}) 

In [614]:
def createLocalHTML(soup, title):
    ## saves images if any image is present in module
    global images_counter
    for src in soup.findAll('img'):
        images_counter += 1
        img_path = f'{images_path}/{images_counter}.jpg'
        with open(f'{images_path}/{images_counter}.jpg', 'wb') as f:
            f.write(requests.get(src['src']).content) 
        src['src'] = img_path
    
    with open(f'{courseTitle}/{title}.html', 'w') as f:
        f.write(str(soup))
createLocalHTML(soup, title) 

In [617]:
with open('vid.mp4', 'wb') as f:
    f.write(requests.get('https://d3c33hcgiwev3.cloudfront.net/2014-08-Guido-Hello-World-02.3179ce70a05811e481fc51ae55819a7c/full/720p/index.mp4?Expires=1609545600&Signature=OhalNsEaKX-S9q76gLVKnm15jYCUhJ~OrGfse1Er7YcQOEEYg6ypJfVovvgJLdp6MCYHJ3mjEiMKEp~ZAZHvLiIN2xCLqZ-AY7llcrSHuhiOcqeqrvgYjXAVf6yeuIrEg5NggqzYHjGwqpyDO4rjsNYwq5qB-ToaAD3ZOW2tdHE_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A').content)